In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np 
import pandas as pd 
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torchvision import datasets, transforms

import warnings
warnings.filterwarnings('ignore')
seed = np.random.seed(100)

In [2]:
#Question 1
#Define functions
def training_loop(n_epochs, optimizer, model, loss_fn, in_train, in_val, out_train, out_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(in_train) # <1>
        loss_train = loss_fn(p_train, out_train)

        p_val = model(in_val) # <1>
        loss_val = loss_fn(p_val, out_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            print(datetime.datetime.now(), f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")
            
def binary_map(x):
    return x.map({'yes':1,"no":0})

In [3]:
# load the dataset
housing = pd.DataFrame(pd.read_csv("./Housing.csv")) 
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'price']
varList=['mainroad','guestroom','basement','hotwaterheating','airconditioning', 'prefarea']
input_size = len(num_vars)-1

housing[varList] = housing[varList].apply(binary_map)

scaler = StandardScaler()
# scaler = MinMaxScaler()

#Split data into training and validation sets
df_train, df_test = train_test_split(housing, train_size=0.8, test_size=0.2, random_state=seed)

df_Newtrain = df_train[num_vars]
df_Newtest = df_test[num_vars]

# scaling the data
df_Newtrain[num_vars] = scaler.fit_transform(df_Newtrain[num_vars])
df_Newtest[num_vars] = scaler.fit_transform(df_Newtest[num_vars])

#Create input and output arrays for both training and validation
out_Newtrain = df_Newtrain.pop('price')
in_Newtrain = df_Newtrain
out_Newtest = df_Newtest.pop('price')
in_Newtest = df_Newtest

# convert the data to tensors
in_train = torch.tensor(in_Newtrain.values).float()
in_val = torch.tensor(in_Newtest.values).float()
out_train = torch.tensor(out_Newtrain.values).float().unsqueeze(-1)
out_val = torch.tensor(out_Newtest.values).float().unsqueeze(-1)

In [4]:
#Question 1, part 1
seq_model = nn.Sequential(
            nn.Linear(input_size, 8), #hidden layer 1
            nn.Tanh(),
            nn.Linear(8, 1)) #output layer

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    in_train = in_train,
    in_val = in_val, 
    out_train = out_train,
    out_val = out_val)

2022-03-03 14:52:15.892236 Epoch 1, Training loss 0.9592, Validation loss 0.9232
2022-03-03 14:52:15.897223 Epoch 10, Training loss 0.9464, Validation loss 0.9090
2022-03-03 14:52:15.901212 Epoch 20, Training loss 0.9326, Validation loss 0.8937
2022-03-03 14:52:15.908194 Epoch 30, Training loss 0.9192, Validation loss 0.8789
2022-03-03 14:52:15.913180 Epoch 40, Training loss 0.9063, Validation loss 0.8645
2022-03-03 14:52:15.917169 Epoch 50, Training loss 0.8937, Validation loss 0.8505
2022-03-03 14:52:15.922156 Epoch 60, Training loss 0.8814, Validation loss 0.8369
2022-03-03 14:52:15.927143 Epoch 70, Training loss 0.8696, Validation loss 0.8237
2022-03-03 14:52:15.932129 Epoch 80, Training loss 0.8580, Validation loss 0.8109
2022-03-03 14:52:15.937118 Epoch 90, Training loss 0.8468, Validation loss 0.7984
2022-03-03 14:52:15.942104 Epoch 100, Training loss 0.8358, Validation loss 0.7863
2022-03-03 14:52:15.947091 Epoch 110, Training loss 0.8252, Validation loss 0.7745
2022-03-03 14:5

In [5]:
#Question 1, part 2
seq_model_2 = nn.Sequential(
            nn.Linear(input_size, 8), #hidden layer 1
            nn.Tanh(),
            nn.Linear(8, 4), #hidden layer 2
            nn.Tanh(),
            nn.Linear(4, 1)) #output layer


optimizer = optim.SGD(seq_model_2.parameters(), lr=1e-3)

training_loop(
    n_epochs = 200, 
    optimizer = optimizer,
    model = seq_model_2,
    loss_fn = nn.MSELoss(),
    in_train = in_train,
    in_val = in_val, 
    out_train = out_train,
    out_val = out_val)

2022-03-03 14:52:16.003946 Epoch 1, Training loss 1.0192, Validation loss 1.0279
2022-03-03 14:52:16.012916 Epoch 10, Training loss 1.0037, Validation loss 1.0110
2022-03-03 14:52:16.019934 Epoch 20, Training loss 0.9873, Validation loss 0.9931
2022-03-03 14:52:16.026877 Epoch 30, Training loss 0.9717, Validation loss 0.9759
2022-03-03 14:52:16.033859 Epoch 40, Training loss 0.9568, Validation loss 0.9594
2022-03-03 14:52:16.039843 Epoch 50, Training loss 0.9425, Validation loss 0.9437
2022-03-03 14:52:16.046858 Epoch 60, Training loss 0.9288, Validation loss 0.9285
2022-03-03 14:52:16.052808 Epoch 70, Training loss 0.9157, Validation loss 0.9140
2022-03-03 14:52:16.057826 Epoch 80, Training loss 0.9031, Validation loss 0.9000
2022-03-03 14:52:16.063778 Epoch 90, Training loss 0.8910, Validation loss 0.8865
2022-03-03 14:52:16.069761 Epoch 100, Training loss 0.8793, Validation loss 0.8734
2022-03-03 14:52:16.075746 Epoch 110, Training loss 0.8681, Validation loss 0.8609
2022-03-03 14:5

In [6]:
#Question 2:

#Load and preprocess dataset
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

cifar10_train = [(img, label) for img, label in cifar10]
cifar10_test = [(img, label) for img, label in cifar10_val]

#Use GPU if available
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

Files already downloaded and verified
Files already downloaded and verified
cuda


In [7]:
#Question 2, Part 1:

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)

model_cifar = nn.Sequential(
            nn.Linear(3072, 512), #Hidden Layer 1
            nn.Tanh(),
            nn.Linear(512, 10)) #Output layer

model_cifar.to(device)

learning_rate = 1e-2

optimizer = optim.SGD(model_cifar.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 301

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
      print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch, loss)) 

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Train Accuracy: %f" % (correct / total))

val_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Val Accuracy: %f" % (correct / total))

2022-03-03 14:52:33.691791 Epoch 0, Training loss 1.8294525146484375
2022-03-03 14:52:45.007460 Epoch 10, Training loss 1.5903112888336182
2022-03-03 14:52:56.404122 Epoch 20, Training loss 0.8837257027626038
2022-03-03 14:53:07.802017 Epoch 30, Training loss 0.8088313937187195
2022-03-03 14:53:19.669580 Epoch 40, Training loss 0.31869271397590637
2022-03-03 14:53:33.029318 Epoch 50, Training loss 0.22469337284564972
2022-03-03 14:53:44.976459 Epoch 60, Training loss 0.3721758723258972
2022-03-03 14:53:56.718847 Epoch 70, Training loss 0.2668774127960205
2022-03-03 14:54:08.434106 Epoch 80, Training loss 0.08345070481300354
2022-03-03 14:54:20.243292 Epoch 90, Training loss 0.04918540269136429
2022-03-03 14:54:31.984102 Epoch 100, Training loss 0.03226412460207939
2022-03-03 14:54:43.551045 Epoch 110, Training loss 0.015502438880503178
2022-03-03 14:54:55.152285 Epoch 120, Training loss 0.019649691879749298
2022-03-03 14:55:06.784636 Epoch 130, Training loss 0.025698699057102203
2022-0

In [8]:
#Question 2, Part 1:

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=True)

model_cifar_2 = nn.Sequential(
            nn.Linear(3072, 512), #hidden layer 1
            nn.Tanh(),
            nn.Linear(512, 1024), #hidden layer 2
            nn.Tanh(),
            nn.Linear(1024, 10)) #output layer

model_cifar_2.to(device)

learning_rate = 1e-2

optimizer = optim.SGD(model_cifar_2.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

n_epochs = 301

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar_2(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
      print('{} Epoch {}, Training loss {}'.format(datetime.datetime.now(), epoch, loss))

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar_2(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Train Accuracy: %f" % (correct / total))

val_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model_cifar_2(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Val Accuracy: %f" % (correct / total))

2022-03-03 14:58:30.020766 Epoch 0, Training loss 1.7073450088500977
2022-03-03 14:58:45.645252 Epoch 10, Training loss 1.473246455192566
2022-03-03 14:59:00.630556 Epoch 20, Training loss 1.0912140607833862
2022-03-03 14:59:15.596679 Epoch 30, Training loss 0.5794255137443542
2022-03-03 14:59:30.451327 Epoch 40, Training loss 0.28757891058921814
2022-03-03 14:59:46.059999 Epoch 50, Training loss 0.05835846811532974
2022-03-03 15:00:01.419002 Epoch 60, Training loss 0.02068966068327427
2022-03-03 15:00:18.586125 Epoch 70, Training loss 0.01195518858730793
2022-03-03 15:00:34.404188 Epoch 80, Training loss 0.004421709105372429
2022-03-03 15:00:49.796586 Epoch 90, Training loss 0.006905160844326019
2022-03-03 15:01:04.876555 Epoch 100, Training loss 0.0030110320076346397
2022-03-03 15:01:19.957236 Epoch 110, Training loss 0.0021306113339960575
2022-03-03 15:01:35.237171 Epoch 120, Training loss 0.0025274697691202164
2022-03-03 15:01:50.437061 Epoch 130, Training loss 0.002156391507014632